# Consulta Amigable MEF: Análisis Presupuestal a nivel de Gobiernos Locales

_____________________
```
Proyecto   : WS CAMEF
Script     : d_analysis.ipynb
Descripción: Análisis de data y reporte.
Fecha      : 2025-02-15
Versión    : 1.0 (draft)
Autor      : Alex Evanan & Mauricio Beuzeville

H.Revisión:
    - [2025-02-07] v1.0: Versión incial.
    - [2025-03-02] V1.1: Gráficos y tablas
```
_____________________

## Librerías

In [48]:
# Librerias
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from matplotlib.ticker import StrMethodFormatter


## Carga de Datos

In [49]:
# Directorio de trabajo
dir_path          = os.getcwd()
dir_path_data_pro = os.path.join(os.path.dirname(dir_path), "01_data/02_processed")
dir_path_results  = os.path.join(os.path.dirname(dir_path), "04_results/01_graphs")

# Archivos
file_muni = "PROCESADO_EJECUCION_GASTO_GL_X_MUNICIPALIDADES.xlsx"

# Carga de datos
df = pd.read_excel(
    os.path.join(dir_path_data_pro, file_muni), dtype={col: str for col in range(1, 8)}
)
            
# Información del dataframe
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20674 entries, 0 to 20673
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Año                             20674 non-null  int64  
 1   UBI_DPTO                        20674 non-null  object 
 2   Departamento                    20674 non-null  object 
 3   UBI_PROV                        20674 non-null  object 
 4   Provincia                       20674 non-null  object 
 5   UBI_DIST                        20674 non-null  object 
 6   COD_SIAF                        20674 non-null  object 
 7   Municipalidad                   20674 non-null  object 
 8   PIA                             20674 non-null  int64  
 9   PIM                             20674 non-null  int64  
 10  Certificación                   20674 non-null  int64  
 11  Compromiso Anual                20674 non-null  int64  
 12  Atención de Compromiso Mensual  

## Análisis

In [50]:
# Procesamiento de datos

# Ajuste de etiquetas
df = df.replace('PROVINCIA CONSTITUCIONAL DEL CALLAO',"CALLAO")

df.head()


,Año,UBI_DPTO,Departamento,UBI_PROV,Provincia,UBI_DIST,COD_SIAF,Municipalidad,PIA,PIM,Certificación,Compromiso Anual,Atención de Compromiso Mensual,Devengado,Girado,Avance %
0,2015,01,AMAZONAS,0101,CHACHAPOYAS,010101,300001,MUNICIPALIDAD PROVINCIAL DE CHACHAPOYAS,11099547,72422840,67817314,45827779,45827779,45820914,45811893,63.3
1,2015,01,AMAZONAS,0101,CHACHAPOYAS,010102,300002,MUNICIPALIDAD DISTRITAL DE ASUNCION,370554,1089344,387442,395018,395018,395018,392618,36.3
2,2015,01,AMAZONAS,0101,CHACHAPOYAS,010103,300003,MUNICIPALIDAD DISTRITAL DE BALSAS,403348,598968,581088,581088,581088,581064,581064,97.0
3,2015,01,AMAZONAS,0101,CHACHAPOYAS,010104,300004,MUNICIPALIDAD DISTRITAL DE CHETO,394983,1899721,1859456,694472,694472,694472,694472,36.6
4,2015,01,AMAZONAS,0101,CHACHAPOYAS,010105,300005,MUNICIPALIDAD DISTRITAL DE CHILIQUIN,401805,1252502,533946,526948,465044,465044,465043,37.1


### PIM, Devengado y Avance

In [51]:
# Grafico PIM, Devengado y Avance

# Data para el gráfico
df_gph = (
    df.loc[df["Año"] <= 2025]
    .groupby("Año", as_index=False)
    .agg({
        "PIM": lambda x: x.sum() / 1_000_000,
        "Devengado": lambda x: x.sum() / 1_000_000,
        "Avance %": "mean"
    })
).copy()

df_gph.round(2)

,Año,PIM,Devengado,Avance %
0,2015,28902.33,21098.69,74.39
1,2016,31373.91,22980.62,74.28
2,2017,34902.69,24134.20,73.25
3,2018,37850.76,27091.54,75.22
4,2019,35634.44,24668.12,72.48
5,2020,42727.58,27057.78,67.54
6,2021,45000.85,32231.19,74.68
7,2022,52860.45,37737.23,76.88
8,2023,49714.72,36100.96,75.79
9,2024,51766.26,39369.36,71.56


En el período **2015-2024**, el avance en la **ejecución presupuestal** de los gobiernos locales alcanzó su punto más bajo en **2020**, con un **68%**. Por otro lado, el **2022** registró el mayor avance, con un **77%**, evidenciando una mejora en la gestión y ejecución del presupuesto. En promedio, la ejecución presupuestal a nivel de municipalidades es de **73%**. Es decir 27 de cada 100 soles son devueltos al MEF.

![](../04_results/01_graphs/gph_03a.png)
<!-- <img src="../04_results/01_graphs/gph_03a.png" alt="Gráfico" width="600" height="auto"> -->


### Participación PIM Departamental

In [52]:
# Grafico PIM por departamento v2

# Data para el gráfico
df_gph = df.loc[df["Año"] <= 2025].pivot_table(
    index  =["Departamento"],
    columns=["Año"],
    values =["PIM"],
    aggfunc={
        "PIM": lambda x: x.sum() / 1_000_000,
        },
    # fill_value=0,
    margins=False,
    margins_name="Total",
    dropna=True,
    observed=True,
).copy()

# Eliminar el nivel 0
df_gph.columns = df_gph.columns.droplevel(0)

# Columna promedio
df_gph["Prom."] = df_gph.mean(axis=1)

# ordenar segun promedio
df_gph = df_gph.sort_values(by="Prom.", ascending=False)

# Transformar en porcentajes
df_gph = round(df_gph.div(df_gph.sum(axis=0), axis=1) * 100,2)

df_gph

Año,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,Prom.
Departamento,,,,,,,,,,,,
LIMA,20.68,20.66,21.73,22.64,20.50,18.23,17.92,15.44,15.83,18.88,21.03,19.08
CUSCO,9.23,8.16,7.71,8.65,8.71,7.36,8.22,10.47,12.06,10.17,9.96,9.30
ANCASH,4.45,4.50,5.19,6.34,6.95,7.14,8.17,9.68,9.27,8.71,7.63,7.41
PIURA,5.83,5.52,5.27,5.69,6.87,6.94,7.03,6.63,7.32,6.35,5.88,6.38
AREQUIPA,5.62,5.55,5.34,6.09,5.84,6.21,6.69,7.33,6.45,6.05,6.22,6.21
LA LIBERTAD,5.32,5.40,5.53,5.99,6.99,6.65,6.25,5.46,5.20,4.77,4.76,5.64
CAJAMARCA,5.80,6.24,6.16,5.15,5.43,6.11,6.22,5.62,5.22,4.82,4.85,5.56
PUNO,4.93,4.78,4.47,3.60,3.41,4.06,3.62,3.53,3.44,3.25,2.98,3.74
JUNIN,3.86,3.96,3.88,3.79,3.62,3.76,3.40,3.28,3.32,3.30,3.27,3.54


Según la distribución porcentual del **PIM** (a nivel de gobiernos locales) por departamentos (2015-2025), **Lima** lidera con un promedio del **19.08%**, aunque su participación ha disminuido entre 2018-2023. Mientras que **Madre de Dios** y **Tumbes** tienen la menor participación con menos de **1%** en promedio.

![](../04_results/01_graphs/gph_04b.png)

### Relación N° Municipalidades y PIM

In [53]:
# Tabla Num Municipalidades

# Data para el gráfico
df_gph = df.loc[df["Año"] == 2025].pivot_table(
    index  =["Departamento"],
    columns=["Año"],
    values =[
        "Municipalidad",
        # "Municipalidad",
        ],
    aggfunc={
        "Municipalidad": [
            lambda x: x.str.contains("PROVINCIAL|METROPOLITANA").sum(),  # Contar "PROVINCIAL"
            lambda x: x.str.contains("DISTRITAL").sum(),   # Contar "DISTRITAL"
        ]
        },
    # fill_value=0,
    margins=False,
    margins_name="Total",
    dropna=True,
    observed=True,
).copy()

# Eliminar el nivel 0
df_gph.columns = df_gph.columns.droplevel(0)

#Renombrar columnas
df_gph.columns = ["Muni. Provincial", "Muni. Distrital"]

# ordenar segun promedio
df_gph = df_gph.sort_values(by="Muni. Distrital", ascending=False)

# Agregar totales de filas y columnas
df_gph = df_gph.assign(Total=df_gph.sum(axis=1))  # Totales de filas

df_gph

,Muni. Provincial,Muni. Distrital,Total
Departamento,,,
LIMA,10,161,171
ANCASH,20,146,166
JUNIN,9,115,124
CAJAMARCA,13,114,127
AYACUCHO,11,113,124
CUSCO,13,103,116
AREQUIPA,8,101,109
PUNO,13,97,110
HUANCAVELICA,7,95,102


Si bien **Lima** lidera la asignación del PIM con un promedio del **19.08%**, el gráfico de dispersión muestra que también es el departamento con más distritos. Aunque se observa una **tendencia positiva** entre el **número de distritos** y el **PIM promedio**, esta relación no es proporcional en todos los casos. Esto sugiere que, si bien la cantidad de distritos influye en la asignación del PIM, no es el único factor determinante.

![](../04_results/01_graphs/gph_05a.png)

### PIM por Distritos de Lima Metropolitana

In [54]:
# Gráfico PIM por Distritos de Lima Met.

# Data para el gráfico
df_gph = df.loc[
    (df["Año"] <= 2024)&
    (df["Provincia"]=="LIMA")&
    (df["Municipalidad"].str.contains("DISTRITAL"))
    ].pivot_table(
    index  =["Municipalidad"],
    columns=["Año"],
    values =[
        "PIM",
        # "Avance %",
        ],
    aggfunc={
        "PIM": lambda x: x.sum() / 1_000_000,
        # "Avance %": "mean",
        },
    # fill_value=0,
    margins=False,
    margins_name="Total",
    dropna=True,
    observed=True,
).copy()


# Eliminar el nivel 0
df_gph.columns = df_gph.columns.droplevel(0)

# Columna promedio
df_gph["Prom."] = df_gph.mean(axis=1)

# ordenar segun promedio
df_gph = df_gph.sort_values(by="Prom.", ascending=False)

df_gph

Año,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,Prom.
Municipalidad,,,,,,,,,,,
MUNICIPALIDAD DISTRITAL DE SANTIAGO DE SURCO,241.101572,265.395153,273.491925,298.191681,317.180675,279.479249,340.752050,349.934636,428.857206,385.274972,317.965912
MUNICIPALIDAD DISTRITAL DE ATE - VITARTE,217.467838,259.955380,296.205306,314.230074,256.975909,308.797277,297.881138,284.817998,245.437917,320.409970,280.217881
MUNICIPALIDAD DISTRITAL DE SAN ISIDRO,211.389121,238.776481,235.450881,252.874501,289.791456,287.944793,299.718423,307.791203,288.304251,339.603985,275.164510
MUNICIPALIDAD DISTRITAL DE SAN JUAN DE LURIGANCHO,170.500676,228.024979,227.348237,221.157253,198.883182,283.559829,269.231439,312.404070,366.439754,446.618478,272.416790
MUNICIPALIDAD DISTRITAL DE MIRAFLORES,193.129622,197.758827,204.730412,227.623191,268.845661,232.182006,253.149548,285.553658,305.639797,370.719549,253.933227
MUNICIPALIDAD DISTRITAL DE SAN MARTIN DE PORRES,128.088269,135.340928,155.422292,168.870036,157.016910,195.850676,166.749143,168.232618,188.544540,250.844695,171.496011
MUNICIPALIDAD DISTRITAL DE LA MOLINA,123.322621,155.263349,183.374077,180.301459,159.474578,196.123899,162.382560,172.401958,147.102412,150.101752,162.984866
MUNICIPALIDAD DISTRITAL DE VILLA EL SALVADOR,115.305158,123.584245,180.656346,123.830894,137.873181,167.576310,181.143429,156.342875,184.933214,203.605168,157.485082
MUNICIPALIDAD DISTRITAL DE PUENTE PIEDRA,117.044155,125.280270,142.225616,164.269271,143.269287,145.843064,156.043968,161.401068,170.921034,179.455898,150.575363


Distritos como **Santiago de Surco**, **Ate-Vitarte** y **San Isidro** tienen los PIM más altos, con promedios de **318**, **280** y **275** millones de soles, respectivamente. En contraste, distritos como **Santa María del Mar**, **Santa Rosa** y **San Bartolo** registran los PIM más bajos, con promedios inferiores a **14** millones de soles.

![](../04_results/01_graphs/gph_06a.png)

Al aplicar el método de **agrupamiento jerárquico** (Ward Jr), se identifican con mayor claridad **grupos de distritos** con asignaciones presupuestarias similares a lo largo del tiempo. Algunos presentan niveles persistentemente elevados, mientras que otros comparten características de menor magnitud. La estructura del **dendrograma** (eje izquierdo) sugiere al menos **7 grupos** bien definidos y revela la proximidad entre ellos. Curiosamente, dentro de estos grupos se observa una **tendencia** a la aglomeración de algunos distritos según su **ubicación geográfica** como **Lima centro, norte y sur**.

![](../04_results/01_graphs/gph_06b.png)

### Ejecuación por Distritos de Lima Metropolitana

In [55]:
# Grafico Ejecución Distritos de Lima Meretropolitana V2

# Data para el gráfico
df_gph = df.loc[
    (df["Año"] <= 2024)&
    (df["Provincia"]=="LIMA")&
    (df["Municipalidad"].str.contains("DISTRITAL"))
    ].pivot_table(
    index  =["Municipalidad"],
    columns=["Año"],
    values =[
        "Avance %",
        ],
    aggfunc={
        "Avance %": "mean",
        },
    # fill_value=0,
    margins=False,
    margins_name="Total",
    dropna=True,
    observed=True,
).copy()

# Eliminar el nivel 0
df_gph.columns = df_gph.columns.droplevel(0)

# Columna promedio
df_gph["Prom."] = df_gph.mean(axis=1)

# ordenar segun promedio
df_gph = df_gph.sort_values(by="Prom.", ascending=False)

# Transformar en porcentajes
df_gph = round(df_gph,1)

df_gph

Año,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,Prom.
Municipalidad,,,,,,,,,,,
MUNICIPALIDAD DISTRITAL DE JESUS MARIA,80.7,85.4,85.6,90.7,90.8,87.8,87.3,91.5,93.7,94.3,88.8
MUNICIPALIDAD DISTRITAL DE MIRAFLORES,94.0,92.7,95.7,94.9,83.2,89.1,85.1,80.6,62.4,90.9,86.9
MUNICIPALIDAD DISTRITAL DE CARABAYLLO,92.8,86.9,89.0,71.5,86.3,82.1,95.4,91.2,81.9,85.8,86.3
MUNICIPALIDAD DISTRITAL DE SAN MIGUEL,85.9,78.7,83.1,79.8,85.5,80.2,85.1,92.3,91.7,93.9,85.6
MUNICIPALIDAD DISTRITAL DE MAGDALENA DEL MAR,82.5,94.4,77.8,71.3,82.7,79.2,92.8,89.3,90.7,92.9,85.4
MUNICIPALIDAD DISTRITAL DE SAN BORJA,85.4,86.7,90.7,82.5,78.3,74.6,83.6,92.4,82.8,89.1,84.6
MUNICIPALIDAD DISTRITAL DE SANTIAGO DE SURCO,85.9,82.0,82.3,86.3,77.0,86.8,74.7,82.2,87.7,96.1,84.1
MUNICIPALIDAD DISTRITAL DE PUEBLO LIBRE,80.4,75.3,68.6,78.6,90.8,84.5,89.5,86.7,92.1,93.3,84.0
MUNICIPALIDAD DISTRITAL DE LURIGANCHO (CHOSICA),84.7,88.3,76.0,78.2,72.4,73.4,81.6,93.6,92.8,98.7,84.0


En cuanto a la **ejecución presupuestal**, distritos como **Jesús María, Miraflores y Carabayllo** registraron los mayores porcentajes, superando el **86%** en promedio. En contraste, **San Luis, El Agustino y Villa María del Triunfo** presentaron niveles inferiores al **72%**, aunque han mostrado una tendencia de crecimiento en los últimos años.

![](../04_results/01_graphs/gph_07b.png)

### Ranking Distritos de Lima Metropolitana

In [56]:
# Grafico Ranking de Distritos de Lima Meretropolitana por PIM vs Avance V2

# Data para el gráfico
df_gph = df.loc[
    (df["Año"] == 2024)&
    (df["Provincia"]=="LIMA")&
    (df["Municipalidad"].str.contains("DISTRITAL"))
    ].pivot_table(
    index  =["Municipalidad"],
    # columns=["Año"],
    values =[
        "PIM",
        "Avance %",
        ],
    aggfunc={
        "PIM": lambda x: x.sum() / 1_000_000,
        "Avance %": "mean",
        },
    # fill_value=0,
    margins=False,
    margins_name="Total",
    dropna=True,
    observed=True,
).copy()

# ordenar segun promedio
df_gph = df_gph.sort_values(by="PIM", ascending=True).reset_index().round(1)

# Ranking
df_gph["Rank. PIM"] = df_gph["PIM"].rank(ascending=False, method="first").astype(int)
df_gph["Rank. Avance"] = df_gph["Avance %"].rank(ascending=False, method="first").astype(int)

df_gph

,Municipalidad,Avance %,PIM,Rank. PIM,Rank. Avance
0,MUNICIPALIDAD DISTRITAL DE SANTA MARIA DEL MAR,93.0,10.7,42,8
1,MUNICIPALIDAD DISTRITAL DE SAN BARTOLO,81.3,15.5,41,34
2,MUNICIPALIDAD DISTRITAL DE PUNTA NEGRA,84.4,18.2,40,28
3,MUNICIPALIDAD DISTRITAL DE PUCUSANA,80.4,24.9,39,36
4,MUNICIPALIDAD DISTRITAL DE SANTA ROSA,80.1,26.3,38,37
5,MUNICIPALIDAD DISTRITAL DE CHACLACAYO,81.5,27.9,37,33
6,MUNICIPALIDAD DISTRITAL DE ANCON,89.0,32.8,36,16
7,MUNICIPALIDAD DISTRITAL DE SAN LUIS,87.5,34.4,35,19
8,MUNICIPALIDAD DISTRITAL DE CIENEGUILLA,76.3,39.1,34,38
9,MUNICIPALIDAD DISTRITAL DE PUNTA HERMOSA,81.8,46.0,32,32


Finalmente, al rankear los distritos por PIM en **2024** y compararlos con su nivel de ejecución presupuestal, se observa que **Villa María del Triunfo, Villa El Salvador** y **San Isidro**, a pesar de estar en el **top 10** de los distritos con mayor asignación de PIM, se encuentran entre los **10 últimos** en el ranking de avance de ejecución.

Por otro lado, **Santa María del Mar, Magdalena del Mar** y **Pueblo Libre** lograron ubicarse en el **top 10** de ejecución presupuestal, a pesar de contar con una asignación de PIM inferior a la mitad de la de los distritos con mayor presupuesto.

![](../04_results/01_graphs/gph_08b.png)
<!-- <img src="../04_results/01_graphs/gph_08b.png" alt="Gráfico" width="600" height="auto"> -->


Estos resultados plantean interrogantes fundamentales sobre la gestión del presupuesto a nivel distrital.
* ¿Por qué algunos distritos con mayores asignaciones de PIM presentan bajos niveles de ejecución?.

* ¿Podría deberse a una menor **capacidad de absorción del gasto público** cuando enfrentan presupuestos significativamente mayores a los que están acostumbrados a manejar?.

* ¿**Mayores asignaciones** pueden dispersar su presupuesto en múltiples iniciativas, aumentando el **riesgo de retrasos o inejecución**?

* ¿Existe mas burocracia o deficiencias en la planificación?.

Por otro lado, los distritos con menor presupuesto han logrado niveles de ejecución más eficientes. 
* ¿Qué estrategias han implementado para optimizar sus recursos? 

* ¿Acaso la **urgencia en el gasto** los lleva a priorizar proyectos esenciales y ejecutarlos con mayor rapidez?

Una **alta ejecución** presupuestal no siempre es sinónimo de **eficiencia**, así como una **baja ejecución** no implica necesariamente **mala gestión**. Detrás de ambos escenarios pueden existir riesgos y desafíos. 

* Una rápida ejecución podría estar enfocada en proyectos de menor relevancia o responder a presiones por cumplir metas a corto plazo, ocultando deficiencias en la planificación y falta de supervisión. 

* Una baja ejecución podría evidenciar dificultades administrativas, limitaciones en la capacidad de gestión o retrasos en la implementación de proyectos de alto impacto.

**¿cómo evaluar si el gasto público se traduce en inversiones de alto impacto y beneficios reales para la población?**